In [62]:
print("Hello")

Hello


In [63]:
# Import necessary libraries
import pandas as pd
import numpy as np
import re
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from xgboost import XGBClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import accuracy_score, classification_report
from sklearn.utils import shuffle
from nltk.corpus import stopwords

In [64]:

nltk.download('stopwords')

# Load the data
train = pd.read_csv('../dataset/training.csv')  
test = pd.read_csv('../dataset/test.csv')
val = pd.read_csv('../dataset/validation.csv')

[nltk_data] Downloading package stopwords to /Users/apple/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# # Combine data for consistent preprocessing
# data = pd.concat([train, test, val], axis=0)
# data = shuffle(data, random_state=42)

# # Preprocessing function
# def clean_text(text):
#     text = text.lower()
#     text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove special characters and numbers
#     text = ' '.join(word for word in text.split() if word not in stopwords.words('english'))
#     return text

# data['cleaned_text'] = data['text'].apply(clean_text)

In [68]:
val.head()

,text,label
0,im feeling quite sad and sorry for myself but ...,0
1,i feel like i am still looking at a blank canv...,0
2,i feel like a faithful servant,2
3,i am just feeling cranky and blue,3
4,i can have for a treat or if i am feeling festive,1


In [69]:

# Split back into train, test, and validation
# train = data.iloc[:len(train)]
# test = data.iloc[len(train):len(train)+len(test)]
# val = data.iloc[len(train)+len(test):]

# Prepare TF-IDF vectorizer
vectorizer = TfidfVectorizer(max_features=5000)
X_train = vectorizer.fit_transform(train['text'])
X_test = vectorizer.transform(test['text'])
X_val = vectorizer.transform(val['text'])

In [72]:
y_train = train['label']
y_test = test['label']
y_val = val['label']

In [ ]:
# # -----------------------------
# # 1️⃣ Logistic Regression
# # -----------------------------
# lr = LogisticRegression(max_iter=200)
# lr.fit(X_train, y_train)
# y_pred_lr = lr.predict(X_test)
# print("Logistic Regression:\n", classification_report(y_test, y_pred_lr))

Logistic Regression:
               precision    recall  f1-score   support

           0       0.88      0.92      0.90       586
           1       0.85      0.96      0.90       686
           2       0.88      0.69      0.77       154
           3       0.87      0.77      0.82       276
           4       0.88      0.79      0.83       228
           5       0.89      0.56      0.68        70

    accuracy                           0.87      2000
   macro avg       0.87      0.78      0.82      2000
weighted avg       0.87      0.87      0.86      2000



In [ ]:
# # -----------------------------
# # 2️⃣ Support Vector Classifier
# # -----------------------------
# svc = SVC(kernel='linear', probability=True)
# svc.fit(X_train, y_train)
# y_pred_svc = svc.predict(X_test)
# print("SVC:\n", classification_report(y_test, y_pred_svc))

SVC:
               precision    recall  f1-score   support

           0       0.92      0.92      0.92       586
           1       0.91      0.95      0.93       686
           2       0.87      0.76      0.81       154
           3       0.87      0.86      0.87       276
           4       0.85      0.86      0.86       228
           5       0.93      0.73      0.82        70

    accuracy                           0.90      2000
   macro avg       0.89      0.85      0.87      2000
weighted avg       0.90      0.90      0.90      2000



In [ ]:
# # -----------------------------
# # 3️⃣ Random Forest
# # -----------------------------
# rf = RandomForestClassifier(n_estimators=100)
# rf.fit(X_train, y_train)
# y_pred_rf = rf.predict(X_test)
# print("Random Forest:\n", classification_report(y_test, y_pred_rf))

Random Forest:
               precision    recall  f1-score   support

           0       0.92      0.90      0.91       586
           1       0.88      0.92      0.90       686
           2       0.83      0.73      0.78       154
           3       0.85      0.83      0.84       276
           4       0.83      0.86      0.85       228
           5       0.82      0.76      0.79        70

    accuracy                           0.88      2000
   macro avg       0.85      0.83      0.84      2000
weighted avg       0.88      0.88      0.87      2000



In [ ]:
# # -----------------------------
# # 4️⃣ XGBoost
# # -----------------------------
# xgb = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
# xgb.fit(X_train, y_train)
# y_pred_xgb = xgb.predict(X_test)
# print("XGBoost:\n", classification_report(y_test, y_pred_xgb))

/Users/apple/Desktop/Development/BackAug/venv/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [01:22:04] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBoost:
               precision    recall  f1-score   support

           0       0.95      0.89      0.92       586
           1       0.88      0.92      0.90       686
           2       0.79      0.81      0.80       154
           3       0.86      0.86      0.86       276
           4       0.86      0.86      0.86       228
           5       0.73      0.83      0.78        70

    accuracy                           0.88      2000
   macro avg       0.85      0.86      0.85      2000
weighted avg       0.88      0.88      0.88      2000



In [73]:
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D, Bidirectional, BatchNormalization, Dropout
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import tensorflow as tf


In [76]:
max_words = 10000
max_len = 200

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(train['text'])
X_train_seq = tokenizer.texts_to_sequences(train['text'])
X_test_seq = tokenizer.texts_to_sequences(test['text'])
X_val_seq = tokenizer.texts_to_sequences(val['text'])

X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)
X_val_pad = pad_sequences(X_val_seq, maxlen=max_len)

ann = Sequential()
ann.add(Embedding(max_words, 128, input_length=max_len))
ann.add(SpatialDropout1D(0.2))
ann.add(Bidirectional(LSTM(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)))
ann.add(BatchNormalization())
ann.add(Bidirectional(LSTM(32, dropout=0.2, recurrent_dropout=0.2)))
ann.add(Dense(64, activation='relu'))
ann.add(Dropout(0.3))
ann.add(Dense(len(np.unique(y_train)), activation='softmax'))

ann.compile(loss='sparse_categorical_crossentropy', optimizer=AdamW(learning_rate=1e-3), metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-5)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = ann.fit(X_train_pad, y_train, epochs=20, batch_size=64, validation_data=(X_val_pad, y_val), callbacks=[reduce_lr, early_stopping])

y_pred_ann = np.argmax(ann.predict(X_test_pad), axis=1)
print("ANN:\n", classification_report(y_test, y_pred_ann))


Epoch 1/20


/Users/apple/Desktop/Development/BackAug/venv/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


250/250 ━━━━━━━━━━━━━━━━━━━━ 76s 292ms/step - accuracy: 0.3287 - loss: 1.6135 - val_accuracy: 0.4595 - val_loss: 1.5208 - learning_rate: 0.0010
Epoch 2/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 85s 339ms/step - accuracy: 0.5720 - loss: 1.1286 - val_accuracy: 0.7920 - val_loss: 0.5886 - learning_rate: 0.0010
Epoch 3/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 81s 325ms/step - accuracy: 0.8422 - loss: 0.4415 - val_accuracy: 0.9110 - val_loss: 0.2677 - learning_rate: 0.0010
Epoch 4/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 89s 355ms/step - accuracy: 0.9340 - loss: 0.2095 - val_accuracy: 0.9235 - val_loss: 0.2341 - learning_rate: 0.0010
Epoch 5/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 83s 331ms/step - accuracy: 0.9537 - loss: 0.1395 - val_accuracy: 0.9245 - val_loss: 0.2408 - learning_rate: 0.0010
Epoch 6/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 84s 337ms/step - accuracy: 0.9634 - loss: 0.1061 - val_accuracy: 0.9220 - val_loss: 0.2441 - learning_rate: 0.0010
Epoch 7/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 87s 350ms/step - accuracy: 0.9696 - loss:

In [77]:
import pickle

# Save the model to a pickle file
with open('../pickles/model.pkl', 'wb') as file:
    pickle.dump(ann, file)

In [78]:
# Save vectorizer
with open('../pickles/vectorizer.pkl', 'wb') as file:
    pickle.dump(vectorizer, file)


In [86]:
with open('../pickles/tokenizer.pkl', 'wb') as file:
    pickle.dump(tokenizer, file)

In [82]:
arr = [
    "i didnt feel humiliated",
    "i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake",
    "im grabbing a minute to post i feel greedy wrong",
    "i am ever feeling nostalgic about the fireplace i will know that it is still on the property",
    "i am feeling grouchy",
    "ive been feeling a little burdened lately wasnt sure why that was",
    "ive been taking or milligrams or times recommended amount and ive fallen asleep a lot faster but i also feel like so funny",
    "i feel as confused about life as a teenager or as jaded as a year old man",
    "i have been with petronas for years i feel that petronas has performed well and made a huge profit",
    "i feel romantic too",
    "i feel like i have to make the suffering i m seeing mean something",
    "i do feel that running is a divine experience and that i can expect to have some type of spiritual encounter",
    "i think it s the easiest time of year to feel dissatisfied",
    "i feel low energy i m just thirsty",
    "i have immense sympathy with the general point but as a possible proto writer trying to find time to write in the corners of life and with no sign of an agent let alone a publishing contract this feels a little precious",
    "i do not feel reassured anxiety is on each side",
    "i didnt really feel that embarrassed",
    "i feel pretty pathetic most of the time",
    "i started feeling sentimental about dolls i had as a child and so began a collection of vintage barbie dolls from the sixties",
    "i now feel compromised and skeptical of the value of every unit of work i put in",
    "i feel irritated and rejected without anyone doing anything or saying anything",
    "i am feeling completely overwhelmed i have two strategies that help me to feel grounded pour my heart out in my journal in the form of a letter to god and then end with a list of five things i am most grateful for",
    "i have the feeling she was amused and delighted",
    "i was able to help chai lifeline with your support and encouragement is a great feeling and i am so glad you were able to help me",
    "i already feel like i fucked up though because i dont usually eat at all in the morning",
    "i still love my so and wish the best for him i can no longer tolerate the effect that bm has on our lives and the fact that is has turned my so into a bitter angry person who is not always particularly kind",
    "i feel so inhibited in someone elses kitchen like im painting on someone elses picture",
    "i become overwhelmed and feel defeated",
    "i feel kinda appalled that she feels like she needs to explain in wide and lenghth her body measures etc pp",
    "i feel more superior dead chicken or grieving child",
    "i get giddy over feeling elegant in a perfectly fitted pencil skirt",
    "i remember feeling acutely distressed for a few days",
    "i have seen heard and read over the past couple of days i am left feeling impressed by more than a few companies",
    "i climbed the hill feeling frustrated that id pretty much paced entirely wrong for this course and that a factor that has never ever hampered me had made such a dent in the day",
    "i can t imagine a real life scenario where i would be emotionally connected enough with someone to feel totally accepted and safe where it it morally acceptable for me to have close and prolonged physical contact and where sex won t be expected subsequently",
    "i am not sure what would make me feel content if anything",
    "i have been feeling the need to be creative",
    "i do however want you to know that if something someone is causing you to feel less then your splendid self step away from them",
    "i feel a bit rude writing to an elderly gentleman to ask for gifts because i feel a bit greedy but what is christmas about if not mild greed",
    "i need you i need someone i need to be protected and feel safe i am small now i find myself in a season of no words",
    "i plan to share my everyday life stories traveling adventures inspirations and handmade creations with you and hope you will also feel inspired",
    "i already have my christmas trees up i got two and am feeling festive which i m sure is spurring me to get started on this book",
    "ive worn it once on its own with a little concealer and for the days im feeling brave but dont want to be pale then its perfect",
    "i feel very strongly passionate about when some jerk off decides to poke and make fun of us",
    "i was feeling so discouraged we are already robbing peter to pay paul to get our cow this year but we cant afford to not get the cow this way",
    "i was feeling listless from the need of new things something different",
    "i lost my special mind but don t worry i m still sane i just wanted you to feel what i felt while reading this book i don t know how many times it was said that sam was special but i can guarantee you it was many more times than what i used in that paragraph did i tell you she was special",
    "i can t let go of that sad feeling that i want to be accepted here in this first home of mine",
    "on a boat trip to denmark"
]

In [84]:
for text in arr:
    text_seq = tokenizer.texts_to_sequences([text])
    text_pad = pad_sequences(text_seq, maxlen=max_len)
    predicted_class = np.argmax(ann.predict(text_pad), axis=1)
    print(predicted_class)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
[0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
[0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
[3]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
[2]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
[3]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
[0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
[5]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
[4]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
[1]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
[2]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
[0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
[1]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
[3]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
[0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
[1]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
[1]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
[0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
[0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
[0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
[4]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
[3]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
[4]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
[1]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/s